# Project 3: Web API's & NLP

## Collecting the Data with Pushshift

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time

In [2]:
# file paths

input_path = '../data/input/'
output_path = '../data/output/'

In [3]:
headers = {'User-agent':'SomeDomStuff'}

In [4]:
posts = []
subs = []
p_id = []
text = []

This function returns 25 posts per run. It will be ran twice along with the second cell to retrieve 5000 posts from each subreddit. 

In [8]:
# https://www.geeksforgeeks.org/args-kwargs-python/
def get_submission(**kwargs):
   # r = requests.get("https://api.pushshift.io/reddit/search/submission/?subreddit=PS4",params=kwargs)
    r = requests.get("https://api.pushshift.io/reddit/search/submission/?subreddit=XboxOne",params=kwargs)
        
    data = r.json()
    return data['data']

In [9]:
before = None
num_post = 200

# Runs loop while number of collected post is under 5000 
while num_post != 0:
    submissions = get_submission(before=before,sort='desc',sort_type='created_utc')
    if not submissions: break
        
    for submission in submissions:
        before = submission['created_utc'] # This keeps track of position for the next call in the while loop

        # append each comment object to its appropriate list
        p_id.append(submission['id'])
        subs.append(submission['subreddit'])
        if 'selftext' in submission:
            text.append(submission['selftext'])
        else:
            text.append('')
        posts.append(submission['title'])


    #time.sleep(1) # pauses for a second between each 25 request
    time.sleep(.5)
    num_post -= 1 # counts down for while loop

In [10]:
# Checks number of commenet objects recived in list
print(len(posts))
print(len(subs))
print(len(p_id))
print(len(text))

10000
10000
10000
10000


In [11]:
# Create data frame with columns of subreddit title, subreddit, and id
df = pd.DataFrame({'title':posts,'selftext':text,'subreddit':subs,'id': p_id})

In [12]:
# Check to make sure there are no repeated post collected
len(df['id'].unique())

10000

In [13]:
# Create Binary y column based on subreddit name
df['sub'] = df['subreddit'].map(lambda x: 1 if x == 'PS4'else 0)

In [14]:
df

,title,selftext,subreddit,id,sub
0,Medievil Remake Pushing Base PS4 Hard,As a long time fan of the original I have some...,PS4,k6gxvi,1
1,Persona 5 Strikers to Release in the West on F...,,PS4,k6gxei,1
2,Let's Be Friends - /r/PS4 Weekly Friend Finder...,**Weekly Friend Finder and LFG Thread** ([prev...,PS4,k6gp43,1
3,4 of u/Azlaars' incredible PS4 Exclusive poste...,,PS4,k6gooa,1
4,"Hi there,",[removed],PS4,k6gj9z,1
...,...,...,...,...,...
9995,Keep playing on a really old 1080p TV and wait...,[deleted],xboxone,juyrvx,0
9996,My Ford GT-40 controller stand,[deleted],xboxone,juyrfk,0
9997,My home Xbox.,[removed],xboxone,juyqxq,0
9998,Child restrictions enabled without me enabling...,[removed],xboxone,juyp4p,0


In [15]:
df.describe()

,sub
count,10000.000000
mean,0.500000
std,0.500025
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [16]:
df.dtypes

title        object
selftext     object
subreddit    object
id           object
sub           int64
dtype: object

In [16]:
df.isnull().sum()

title        0
selftext     0
subreddit    0
id           0
sub          0
dtype: int64

In [17]:
len(df[df['title']==''])

0

In [18]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool

In [19]:
df['subreddit'].value_counts()

xboxone    5000
PS4        5000
Name: subreddit, dtype: int64

In [20]:
# Checking for duplicated data
df['id'].duplicated().sum()

0

In [21]:
# Dropping any duplicated posts
df_clean = df.drop_duplicates()

In [22]:
df_clean.shape

(10000, 5)

In [24]:
# Save data frame to csv
#df_clean.to_csv('reddit_data1.csv', index=False)